In [1]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd

# SQLite Database for Financial Analysis - SQLite and DB Browser 

<!-- PELICAN_BEGIN_SUMMARY -->

After merging and reshaping the datasets, we can use SQLite to safely store, organize and manipulate data in smaller environments.
SQLite is a flexible database that can do real work in real business environments and the SQLite library is also integrated
into a number of popular scripting languages, including Python.
This database tool can also be useful for accountants doing financial analysis.

<!-- PELICAN_END_SUMMARY -->

<img src="/images/SQLflowchart.png" alt="[img: SQL flowchart]" title="SQLite flowchart" />


#### **What is SQLite?**
from "Using SQLite" by Jay A. Kreibich

What is SQLite: In the simplest terms, SQLite is a public-domain software package that provides a
relational database management system, or RDBMS. Relational database systems are
used to store user-defined records in large tables. In addition to data storage and management, a database engine can process complex query commands that combine data
from multiple tables to generate reports and data summaries.
The “Lite” in SQLite does not refer to its capabilities. Rather, SQLite is lightweight
when it comes to setup complexity, administrative overhead, and resource usage

#### **What is DB Browser for SQLite?**
DB Browser for SQLite is a high quality, visual, open source tool used to create, design, and edit database files compatible with SQLite.

It is for users and developers wanting to create, search, and edit databases. It uses a familiar spreadsheet-like interface,
and you don't need to learn complicated SQL commands. This program is not a visual shell for the sqlite command line tool.

#### **Why would we use SQLite to read Excel Files?**
- Sort data on several fields
- Combine data from different spreadheets 
- Use free graphical user interface (GUI) tools to run SQL queries on database
- Convert Excel files to tables
- Install Scientific Python to convert Excel file in SQLite database in just 8 lines of code
- Note: Excel tab name cannot have spaces, but underscores are acceptable

#### **Connecting SQLite **
- The example is an Excel file "PremiumLossSQLite.xlsx" 
- The file has three tabs - premium, loss and reinsurance
- The first row in each sheet should have legal column names. Here is the conversion code:

In [2]:
# Import library
import sqlite3

# Name of Excel xlsx file. SQLite database will have the same name and extension .db
filename="data/PremiumLossSQLite"   

# filename+".db"
# use if_exists so we can repeat the process when new data is available for rerun

con=sqlite3.connect(filename+".db") 
wb=pd.read_excel(filename+'.xlsx',sheet_name=None)
for sheet in wb:
    wb[sheet].to_sql(sheet,con, index=False, if_exists='replace')  
con.commit()

#con.close()    # keep the connection open until you finish all the modification

#### **A handy tool to visualize and access SQLite databases is the free DB Browser**
#### **Below is the screenshot of the database structures after connecting from Excel to SQLite by using Python**
<img src="/images/DBimage.png" alt="[img: DB Browser after import Excel to SQLite and view the table]" title="DbBrowser Query" />

#### ** Query inner join between the reinsurance, premium and loss data and export the data to Excel**
- Query list of policies with incurred losses that covered under the reinsurance treaty
- Instead of Excel VLookup function that has certian limitation
- We can repeat/automate the query process when new data get updated 
- Save the query result to SQLite
- Save the query result to Excel

In [3]:
sql_statement='''
select r.PolicyNumber as Policy_from_Reinsurance_Tab, p.PolicyNumber as Policy_from_Policy_Tab, l.PolicyNumber as Policy_from_Loss_Tab
from reinsurance as r
join premium as p on p.PolicyNumber=r.PolicyNumber
join loss as l on l.PolicyNumber=r.PolicyNumber
'''
df = pd.read_sql(sql_statement,con)

df.to_sql('reinsurace_query_results', con,if_exists='replace') 
df.to_excel('data/reinsurace_query_results.xlsx') 

#### ** Save the query result in SQLite**
<img src="/images/SQLQuery.png" alt="[img: Reinsurance Query Save in SQLite]" title="Reinsurance Query SQL " />

#### **Save the query result as an Excel file**
<img src="/images/reinsurance.jpg" alt="[img: Reinsurance Query Save in Excel from SQLite]" title="Reinsurance Query Excel" />

#### ** Insert a column to table - "premium" **
- Option A - use DML to add a column :-|
- Option B - use to_sql to add a column - much easier and more native to PANDAS way of doing things

In [4]:
# Premium table before inserting new column
sql_statement='''
select p.* from premium as p
'''
df = pd.read_sql(sql_statement,con)
df.head()

,PolicyNumber,FirstName,LastName,PolicyEffectiveDate,LOB,PolicyType,Premium,Coverage,ExcessCoverage,ExcessPolicy
0,10880,Kurtis,Dumm,2017-07-08 00:00:00,Phy,CM,232058,1000000,3900000,96289.0
1,10948,Florencia,Bilyk,2017-06-05 00:00:00,Podiatrist,CM,5132,1000000,3900000,99671.0
2,10962,Taisha,Whack,2017-09-19 00:00:00,Dentist,OCC,11308,1000000,3900000,91809.0
3,11028,Yun,Linely,2017-10-11 00:00:00,Dentist,OCC,13381,1000000,3900000,94102.0
4,10244,Wonda,Hallsworth,2017-08-07 00:00:00,Dentist,OCC,13330,1000000,3900000,92319.0


In [5]:
## Option A - use DML to add a column :-|

# Obtain a Cursor object to execute SQL statements

cur=con.cursor()

# Add a new column to premium table

addColumn = "ALTER TABLE premium ADD COLUMN Reinsurers varchar(32)"

cur.execute(addColumn) 

sql_statement='''
select p.* from premium as p
'''
df = pd.read_sql(sql_statement,con)
df.head()

,PolicyNumber,FirstName,LastName,PolicyEffectiveDate,LOB,PolicyType,Premium,Coverage,ExcessCoverage,ExcessPolicy,Reinsurers
0,10880,Kurtis,Dumm,2017-07-08 00:00:00,Phy,CM,232058,1000000,3900000,96289.0,None
1,10948,Florencia,Bilyk,2017-06-05 00:00:00,Podiatrist,CM,5132,1000000,3900000,99671.0,None
2,10962,Taisha,Whack,2017-09-19 00:00:00,Dentist,OCC,11308,1000000,3900000,91809.0,None
3,11028,Yun,Linely,2017-10-11 00:00:00,Dentist,OCC,13381,1000000,3900000,94102.0,None
4,10244,Wonda,Hallsworth,2017-08-07 00:00:00,Dentist,OCC,13330,1000000,3900000,92319.0,None


In [6]:
## Option B - use to_sql to add a column - much easier and more native to PANDAS way of doing things

df['Reinsurers2']='ABC Re'
df.to_sql('premium',con,if_exists='replace') 

df.head()

,PolicyNumber,FirstName,LastName,PolicyEffectiveDate,LOB,PolicyType,Premium,Coverage,ExcessCoverage,ExcessPolicy,Reinsurers,Reinsurers2
0,10880,Kurtis,Dumm,2017-07-08 00:00:00,Phy,CM,232058,1000000,3900000,96289.0,None,ABC Re
1,10948,Florencia,Bilyk,2017-06-05 00:00:00,Podiatrist,CM,5132,1000000,3900000,99671.0,None,ABC Re
2,10962,Taisha,Whack,2017-09-19 00:00:00,Dentist,OCC,11308,1000000,3900000,91809.0,None,ABC Re
3,11028,Yun,Linely,2017-10-11 00:00:00,Dentist,OCC,13381,1000000,3900000,94102.0,None,ABC Re
4,10244,Wonda,Hallsworth,2017-08-07 00:00:00,Dentist,OCC,13330,1000000,3900000,92319.0,None,ABC Re


In [ ]:
con.close()